In [21]:
def bm25(query, documents,  k=1.2, b=0.75):
    
    def tokenize(s):
        return s.split(" ")
    N = len(documents)
    scores = []
    documents = [tokenize(d) for d in documents]
    avgdl = sum(len(d) for d in documents)/len(documents)
    for document in documents:
        score = 0
        for word in tokenize(query):
            freq = document.count(word)
            tf = (freq * (k + 1)) / (freq + k * (1 - b + b * (len(document) / avgdl)))
            N_q = sum([doc.count(word) for doc in documents])  
            idf = np.log(((N - N_q + 0.5) / (N_q + 0.5)) + 1)
            score += round(tf*idf, 4)
            
        scores.append(score)
    return scores

In [26]:
def get_relevant_sentences(cs, num=10):
    global results

    for pmcid in cs:
        if pmcid.startswith('0'):
            curr_ref_file = pmcid.split('_')[1]
        print(pmcid)
        CURR_PMC = pmcid

        with open(f'Sentences/{curr_ref_file}.json') as f:
            sentences = json.load(f)

        original_sentences = sentences

        results[CURR_PMC] = {}
        idx = 0
        for file, d in cs[pmcid].items():
            if len(d['support']) == 0:
                continue
            claim = d['context']
            idx += 1
            scores = bm25(claim, sentences)
            sorted_scores = np.argsort(scores)[::-1]
            results[CURR_PMC][file] = [original_sentences[i] for i in sorted_scores[:10]]

    

In [27]:
def calculate_recall(cs, results):
    def overlap(start, end, s, e):
        return start <= e and s <= end

    for pmcid in results:
        if pmcid.startswith('0'):
            curr_ref_file = pmcid.split('_')[1]
        with open(f'Articles/{curr_ref_file}.txt') as f:
            content = f.read()
        for file in results[pmcid]:
            indices = [[content.find(s.strip()), content.find(s.strip())+len(s.strip())] for s in results[pmcid][file]]
            count = 0
            support_indices = [[content.find(s.strip()), content.find(s.strip())+len(s.strip())] for s in cs[pmcid][file]['support']]
            for start, end in support_indices:
                for s, e in indices:
                    if overlap(start, end, s, e):
                        count += 1
            cs[pmcid][file]['count'] = count
    res = 0
    total = 0
    for pmcid in cs:
        for file in cs[pmcid]:
            if 'count' in cs[pmcid][file]:
                res += (cs[pmcid][file]['count'] / len(cs[pmcid][file]['support']))
                total += 1

    return res/total

In [28]:
import json
import os
import numpy as np
with open('recon.json') as f:
    cs = json.load(f)

In [29]:
results = {}
get_relevant_sentences(cs)
calculate_recall(cs1, results)

010_PMC3650111
004_PMC3090745
001_PMC2811155
007_PMC3442244
005_PMC3096954
006_PMC3179858
002_PMC2848616
003_PMC2958248
008_PMC3522098
009_PMC3607626


NameError: name 'cs1' is not defined

In [30]:
calculate_recall(cs, results)

0.32601626016260166